In [13]:
import polars as pl
import pandas as pd
from time import time
from pathlib import Path
import warnings

In [15]:
warnings.filterwarnings("ignore")

In [6]:
round((Path("data.csv").stat().st_size / 1048576), 2)

566.11

In [2]:
def elapse_time(func):
    def wrapper(*args, **kwargs):
        start_time = time()
        result = func(*args, **kwargs)
        end_time = time()
        execution_time = round((end_time - start_time), 2)
        print(f"Function {func.__name__} took {execution_time} seconds to execute")
        return result
    return wrapper

In [18]:
@elapse_time
def read_csv(path: str, lib: str) -> pd.DataFrame | pl.DataFrame:
    if lib == "polars":
        df = pl.read_csv(path)
    elif lib == "pandas":
        df = pd.read_csv(path)
    else:
        raise TypeError("lib must be one of 'polars' or 'pandas'.")
    return df

In [19]:
pldf = read_csv("data.csv", "polars")

Function read_csv took 1.19 seconds to execute


In [16]:
pddf = read_csv("data.csv", "pandas")

Function read_csv took 9.97 seconds to execute


In [ ]:
# Schema

In [ ]:
# Describe

In [ ]:
# Backend - Arrow

In [ ]:
# Conditional formatting

In [ ]:
# Joins

In [ ]:
# SQL

In [ ]:
# Exports, scikit learn export, etc.

In [ ]:
@elapse_time
def transform_polars_eager(df: pl.DataFrame) -> pl.DataFrame:
    # TODO

In [ ]:
@elapse_time
def transform_pandas(df: pd.DataFrame) -> pd.DataFrame:
    # TODO

In [ ]:
@elapse_time
def transform_polars_lazy(df: pl.LazyFrame) -> pl.LazyFrame:
    # TODO